# LLM Book Processing Notebook

In [1]:
# 1) OpenAI setup (API key uit .env)

!pip install -q openai python-dotenv

import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("API key not found. Zet OPENAI_API_KEY in je .env")

client = OpenAI(api_key=OPENAI_API_KEY)

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [8]:
# 2) Excel lezen + enriched_description vullen (schrijven naar NIEUW bestand)

!pip install -q pandas openpyxl

import os
import time
import warnings
import pandas as pd

# --- optioneel: warnings onderdrukken ---
os.environ["LC_ALL"] = "C.UTF-8"
os.environ["LANG"] = "C.UTF-8"

warnings.filterwarnings(
    "ignore",
    message="Unknown extension is not supported and will be removed",
    category=UserWarning,
    module="openpyxl"
)

# ====== CONFIG ======
EXCEL_PATH = r"/home/yasin/Git/Cross-domain-recommender/experiment/notebook/complete dataset.xlsx"

# Kies één:
SHEET_NAME = 0         # ✅ eerste sheet als DataFrame
# SHEET_NAME = "Sheet1"  # ✅ specifieke sheetnaam
# SHEET_NAME = None       # ✅ alle sheets (dict) -> code pakt dan automatisch de 1e sheet

NAME_COL = "name"
OVERVIEW_COL = "source_overview"
TYPE_COL = "item_type"
GENRE_COL = "Simplified genre"
ENRICHED_COL = "enriched_description"

MODEL = "gpt-4o-mini"
MAX_RETRIES = 5
SLEEP_BETWEEN_CALLS_SEC = 0.3

# ====== HELPERS ======
def is_missing(v) -> bool:
    if v is None:
        return True
    try:
        if pd.isna(v):
            return True
    except Exception:
        pass
    return str(v).strip() == ""

def make_enrichment_prompt(name: str, item_type: str | None, genre: str | None, overview: str | None) -> str:
    item_type_txt = (item_type or "").strip()
    genre_txt = (genre or "").strip()

    prompt = f"""
Schrijf een verrijkte, informatieve beschrijving (enriched_description) voor dit item.

Naam: "{name}"
Type: "{item_type_txt}"
Simplified genre: "{genre_txt}"

Eisen:
- 120 tot 200 woorden
- Schrijf in het Nederlands
- Objectief en informatief (geen marketingtaal)
- Geen spoilers buiten de hoofdlijn
- Verwerk impliciet: thema's, motieven, toon/sfeer, stijl, karaktertypen, narratieve structuur
- Geen opsomming; 1 samenhangende alinea
""".strip()

    if overview and overview.strip():
        prompt += f"""

Context (source_overview uit dataset):
\"\"\"{overview.strip()}\"\"\"
Gebruik dit als basis, maar verrijk inhoudelijk.
""".rstrip()
    else:
        prompt += "\n\nEr is geen source_overview aanwezig; blijf algemeen en voeg geen specifieke feiten toe die je niet zeker weet."

    return prompt

def call_ai_enriched_description(name: str, item_type: str | None, genre: str | None, overview: str | None) -> str:
    prompt = make_enrichment_prompt(name, item_type, genre, overview)

    last_err = None
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            resp = client.chat.completions.create(
                model=MODEL,
                messages=[
                    {"role": "system", "content": "Geef alleen de enriched_description terug. Geen kopjes, geen extra uitleg."},
                    {"role": "user", "content": prompt},
                ],
                temperature=0.4,
            )
            text = (resp.choices[0].message.content or "").strip()
            if len(text) < 40:
                raise ValueError("AI output too short/empty.")
            return text

        except Exception as e:
            last_err = e
            wait = min(2 ** attempt, 20)
            print(f"[WARN] AI-call faalde (poging {attempt}/{MAX_RETRIES}): {e} → wacht {wait}s")
            time.sleep(wait)

    raise RuntimeError(f"AI-call bleef falen na {MAX_RETRIES} pogingen. Laatste error: {last_err}")

# ====== MAIN ======
raw = pd.read_excel(EXCEL_PATH, sheet_name=SHEET_NAME)

# Als je per ongeluk alle sheets hebt gelezen -> pak de eerste
if isinstance(raw, dict):
    first_sheet_name = next(iter(raw.keys()))
    print(f"sheet_name=None gaf dict terug. Pak eerste sheet: {first_sheet_name}")
    df = raw[first_sheet_name]
else:
    df = raw

print("df type:", type(df))
print("Kolommen:", list(df.columns))

# Check minimale kolommen
for col in [NAME_COL, ENRICHED_COL]:
    if col not in df.columns:
        raise ValueError(f"Kolom ontbreekt: {col}. Beschikbaar: {list(df.columns)}")

# Alleen verrijken waar enriched_description leeg is
to_process = df[df[ENRICHED_COL].apply(is_missing)].index.tolist()
print(f"Rijen te verrijken: {len(to_process)} / {len(df)}")

for i, idx in enumerate(to_process, start=1):
    name = str(df.at[idx, NAME_COL]).strip()
    if not name:
        print(f"[SKIP] Rij {idx}: lege name.")
        continue

    item_type = None if TYPE_COL not in df.columns or is_missing(df.at[idx, TYPE_COL]) else str(df.at[idx, TYPE_COL]).strip()
    genre = None if GENRE_COL not in df.columns or is_missing(df.at[idx, GENRE_COL]) else str(df.at[idx, GENRE_COL]).strip()
    overview = None if OVERVIEW_COL not in df.columns or is_missing(df.at[idx, OVERVIEW_COL]) else str(df.at[idx, OVERVIEW_COL])

    print(f"[{i}/{len(to_process)}] Verrijken: {name}")
    df.at[idx, ENRICHED_COL] = call_ai_enriched_description(name, item_type, genre, overview)

    time.sleep(SLEEP_BETWEEN_CALLS_SEC)

# Opslaan naar NIEUW bestand
OUTPUT_PATH = EXCEL_PATH.replace(".xlsx", "_enriched.xlsx")
df.to_excel(OUTPUT_PATH, index=False)
print("Klaar! Nieuw bestand aangemaakt:", OUTPUT_PATH)



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
df type: <class 'pandas.core.frame.DataFrame'>
Kolommen: ['db_ID', 'source_id', 'item_type', 'name', 'vote_count', 'vote_average', 'source_overview', 'Year', 'source_genres', 'Simplified genre', 'created_by / director / author', 'enriched_description']
Rijen te verrijken: 70 / 140
[1/70] Verrijken: The Long Walk
[2/70] Verrijken: Lost Worlds: Volume 1: Zothique, Averoigne and Others
[3/70] Verrijken: Buy Jupiter and Other Stories
[4/70] Verrijken: Incarnate (Newsoul, #1)
[5/70] Verrijken: Zero Echo Shadow Prime
[6/70] Verrijken: Seize the Fire (Star Trek: Typhon Pact, #2)
[7/70] Verrijken: Emerald Envisage
[8/70] Verrijken: The Doomsday Vault (Clockwork Empire #1)
[9/70] Verrijken: Grasshopper Jungle
[10/70] Verrijken: Tin Woodman
[11/70] Verrijken: The Angel's Game (The Cemetery of Forgotten Books, #2)
[12/70] Verrijken: Monstress, Vol. 1: Awakening (Monstress, #1)
[13/70] Ve